In [4]:
!pip install nltk

In [5]:
import nltk
import re

from nltk.chat.util import Chat, reflections

In [6]:
# Download NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [13]:
# Define patterns and responses
pairs = [
    [r"my name is (.*)", ["Hello %1, how can I assist you today?",]],
    [r"hi|hey|hello", ["Hello, how can I help you?", "Hey there! What can I do for you?",
                       "Hi! How can I assist you today?"]],
    [r"what is your name?", ["I am a chatbot created to assist you. You can call me Chatbot.",]],
    [r"how are you?", ["I'm a bot, so I don't have feelings, but I'm here to help you!",]],
    [r"can you help me with (.*)", ["Sure, I can help you with %1. Please provide more details.",]],
    [r"sorry (.*)", ["It's okay. How can I assist you?",]],
    [r"thank you|thanks", ["You're welcome!", "No problem!", "Happy to help!"]],
    [r"quit", ["Bye! Have a great day!", "Goodbye!"]],
    [r"(.*)", ["I'm sorry, I don't understand that. Can you rephrase?",
               "Could you please elaborate on that?"]],
    [r"(golden temple.*)", ["It is in amritsar,india"]]
]

In [8]:
# Define the chatbot class
class RBChatbot:
    def __init__(self, pairs):
        self.chat = Chat(pairs, reflections)
    def respond(self, user_input):
        return self.chat.respond(user_input)
# Initialize the chatbot
chatbot = RBChatbot(pairs)

In [12]:
def chat_with_bot():
  print("Hi, I'm your chatbot. Type 'quit' to exit.")
  while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
      print("Chatbot: Bye! Have a great day!")
      break
    response = chatbot.respond(user_input)
    print(f"Chatbot: {response}")

chat_with_bot()

Hi, I'm your chatbot. Type 'quit' to exit.
You: my name is srikruthi
Chatbot: Hello srikruthi, how can I assist you today?
You: best place to visit in india
Chatbot: Could you please elaborate on that?
You: how are you doing today
Chatbot: I'm a bot, so I don't have feelings, but I'm here to help you!
You: hello, are you there
Chatbot: Hello, how can I help you?
You: tell me about yourself
Chatbot: I'm sorry, I don't understand that. Can you rephrase?
You: had ur breakfast
Chatbot: Could you please elaborate on that?
You: ok, bye
Chatbot: Could you please elaborate on that?
You: Goodbye!
Chatbot: I'm sorry, I don't understand that. Can you rephrase?
You: can you sing a song for me
Chatbot: I'm sorry, I don't understand that. Can you rephrase?
You: quit
Chatbot: Bye! Have a great day!


In [14]:
!pip install transformers

In [15]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch
# Load pre-trained BERT model and tokenizer
model_name = "deepset/bert-base-cased-squad2"  # BERT trained on SQuAD 2.0
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# Define the context (passage) and question
context = """The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower."""
question = "Who designed the Eiffel Tower?"

# Tokenize the input
inputs = tokenizer(question, context, return_tensors="pt")

# Get model output (start and end index of answer)
with torch.no_grad():
    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

In [17]:
# Get the most probable answer span
start_idx = torch.argmax(start_scores)
end_idx = torch.argmax(end_scores) + 1

# Decode the answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx]))
print(f"Answer: {answer}")

Answer: Gustave Eiffel
